# AI Math Problem Generator

**Import Libraries**

In [1]:
import os
from dotenv import load_dotenv
import json

from openai import OpenAI
from anthropic import Anthropic
from google import generativeai

import gradio as gr
from IPython.display import Markdown, display, update_display

**Load API Keys & Connect**

In [2]:
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')
anthropic_key = os.getenv('ANTHROPIC_API_KEY')
google_key = os.getenv('GOOGLE_API_KEY')

In [3]:
# Connect to the corresponding API's
gpt = OpenAI(api_key = openai_key)
claude = Anthropic(api_key = anthropic_key)
generativeai.configure(api_key = google_key)

**Define System & User Prompts**

In [4]:
# System Prompt
system_prompt = "You are an expert Math Teacher for students ranging from Grade 1 to Grade 12"

In [17]:
# Function for User Prompt with three arguments
def user_prompt(grade_level,num_problems, problem_type, difficulty_level, humour):
    prompt = f"""
    Generate {num_problems} math problems for Grade:{grade_level} and at a {difficulty_level} level for math problems of type:{problem_type}.
    You can also create problems with this characteristic {humour}.
    
    Use the following format:
    {problem_type}
    --------------
    'PROBLEMS:'
        Problem 1. 
        Problem 2.
        ...
        ...
    
    Then, list the solutions similarly in a separate section labeled 'SOLUTIONS:' for each of the problem like shown below:
    'SOLUTION:'
    Solution 1.
    Solution 2.
    ...
    ...
    """
    return prompt

### OpenAI - GPT Models

**Function to call OpenAI LLMS**

In [18]:
def math_problems_gpt(grade_level, num_problems, problem_type,difficulty_level, humour):
    completion = gpt.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [
            {"role":"system", "content": system_prompt},
            {"role":"user", "content": user_prompt(grade_level, num_problems,problem_type,difficulty_level, humour)}
        ]
    )
    result = completion.choices[0].message.content
    return result

In [19]:
print (math_problems_gpt(4, 5,"Decimals-based Algebra","Difficult", "Funny"))

Decimals-based Algebra  
--------------  
'PROBLEMS:'  
Problem 1. Superhero Sam has 5.75 super stickers. He loses 1.25 stickers during a villain battle! How many stickers does Sam have left?  

Problem 2. Tilly the Turtle runs 3.2 meters every day. If Tilly runs for 4.5 days, how many meters does she run in total?  

Problem 3. Mr. Fluffypaws the cat has a collection of 6.8 fish toys. He gives away 2.3 toys to his friends. How many fish toys does Mr. Fluffypaws have now?  

Problem 4. A magical spell costs 4.99 gumballs. If Penny has 20.00 gumballs, how many gumballs does she have left after buying the spell?  

Problem 5. Jenna bought 2.5 liters of rainbow juice for her party. If her friends drink 1.6 liters, how much rainbow juice is left for Jenna to enjoy?  

'SOLUTION:'  
Solution 1. Sam has 5.75 - 1.25 = 4.50 super stickers left.  

Solution 2. Tilly runs 3.2 x 4.5 = 14.40 meters in total.  

Solution 3. Mr. Fluffypaws has 6.8 - 2.3 = 4.5 fish toys left.  

Solution 4. Penny has

**Function to call OpenAI LLM's with streaming**

In [20]:
def stream_math_problems_gpt(grade_level, num_problems,problem_type,difficulty_level, humour):
    completion = gpt.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [
            {"role":"system", "content": system_prompt},
            {"role":"user", "content": user_prompt(grade_level, num_problems,problem_type,difficulty_level, humour)}
        ],
        stream=True
    )


    result = ""
    for text in completion:
        result += text.choices[0].delta.content or ""
        yield result

### Anthropic  Claude Models

**Function to call Claude model**

In [21]:
def math_problems_claude(grade_level, num_problems, problem_type,difficulty_level, humour):
    completion = claude.messages.create(
        model = "claude-3-haiku-20240307",
        max_tokens=1000,
        system = system_prompt,
        messages = [
            {"role":"user", "content": user_prompt(grade_level, num_problems,problem_type,difficulty_level, humour)}
        ]
    )

    result = completion.content[0].text
    return result

**Function to call Claude Models with streaming**

In [22]:
def stream_math_problems_claude(grade_level, num_problems, problem_type,difficulty_level, humour):
    completion = claude.messages.stream(
        model = "claude-3-haiku-20240307",
        max_tokens=1000,
        system = system_prompt,
        messages = [
            {"role":"user", "content": user_prompt(grade_level, num_problems,problem_type,difficulty_level, humour)}
        ]
    )

    result = ""
    with completion as stream:
        for text in stream.text_stream:
            result += text or ""
            yield result

In [23]:
def math_problems(grade_level,num_problems, problem_type,difficulty_level, humour, model):
    if model == "Gpt":
        result = stream_math_problems_gpt(grade_level,num_problems, problem_type,difficulty_level, humour)
    elif model == "Claude":
        result = stream_math_problems_claude(grade_level,num_problems, problem_type,difficulty_level, humour)
        
    for chunk in result:
        yield chunk

### Simple Gradio UI

In [24]:
# All you need is to define the function which u want to call in the interface
# specify the function arguments as inputs: Textboxes, dropdowns etc.
# specify the corresponding Textbox for the output.

In [25]:
# Gradio interface
view = gr.Interface(
    fn=math_problems,
    inputs=[
        gr.Textbox(label = "Grade Level:", value=5),
        gr.Textbox(label="Number of Problems:", value = 5),
        gr.Dropdown(choices=["Decimals-based Algebra", "Algebra", "Word Order Problems", "Geometry Problems" ], 
                    label="Type of Problems",value="Algebra"),
        gr.Dropdown(choices=["Easy", "Medium", "Difficult"], label="Difficulty Level", value="Easy"),
        gr.Dropdown(choices=["Funny", "Scary", "Boring"], label="Tone", value="Boring"),
        gr.Dropdown(choices=["Gpt", "Claude"], label="AI Model", value="Gpt")
    ], # number of inputs here need to match the arguments needed by the function which gradio will use (math
    outputs=[
        gr.Textbox(label = "Result"),
    ],
    allow_flagging="never"
)
view.launch()

C:\Users\sridh\anaconda3\envs\llms\Lib\site-packages\gradio\interface.py:393: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
